In [1]:
import numpy as np
import mediapipe as mp
import cv2
from matplotlib import pyplot as plt
# from IPython.display import cv2_


In [2]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
video_capture = cv2.VideoCapture(0)
video_writter = cv2.VideoWriter('hand_gesture_jepret.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 15, (640, 480))
SAVE_VIDEO = False
# i = 0
with mp_hands.Hands(static_image_mode=False, 
                    max_num_hands=2,
                    min_detection_confidence=0.7) as hands:
    while video_capture.isOpened() and video_writter.isOpened():
        try:
            success, frame = video_capture.read()
            frame_height, frame_width, _ = frame.shape
            # print(frame_width, frame_height)
            if success:
                results = hands.process(cv2.flip(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 1))
                annotated_image = cv2.flip(frame.copy(), 1)
                try:
                    # if not results.multi_hand_world_landmarks:
                    #     continue
                    # for hand_world_landmarks in results.multi_hand_world_landmarks:
                    #     mp_drawing.plot_landmarks(
                    #         hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

                    #     plt.savefig(f'plots/lm_{i}.png')
                    #     i += 1
                    tips = []
                    for hand_landmarks in results.multi_hand_landmarks:
                        finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                        tip = (round(finger_tip.x*frame_width), round(finger_tip.y * frame_height))
                        tips.append(tip)
                        mp_drawing.draw_landmarks(
                            annotated_image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style()) 
                    # if len(tips) == 2:
                    #     annotated_image = cv2.line(annotated_image, 
                    #         tips[0], tips[1],
                    #         (255, 0, 0), 2)
                    # for tip in tips:
                    #     annotated_image = cv2.circle(annotated_image, tip, 10, (255, 0, 0), 2)
                except TypeError:
                    pass
                cv2.imshow('Frame', annotated_image)
                if SAVE_VIDEO:
                    video_writter.write(annotated_image)
            else:
                cv2.imshow('Frame', frame)
                if SAVE_VIDEO:
                    video_writter.write(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(e)
            break
video_capture.release()
video_writter.release()
cv2.destroyAllWindows()